# Homework 1 - Advanced Topics in Macroeconomics
Pedro Augusto Januzzi Guerra

# Questions:

1. Write a function/subroutine with

- Inputs:

    - x = n dimensional real vector
    - f = scalar real function of x

- Outputs:

    - J = n dimensional real Jacobian vector with elements $\partial f(x)/\partial x_i$
    - H = n x n dimensional real Hessian matrix with elements $\partial^2 f(x)/\partial x_i\partial x_j$


2. Write a function/subroutine (as in McGrattan(1990)) with:

- Inputs:
    - A,B,C,Q,R,W (from LQ problem)
- Outputs:
    - F,P (summarizing policy and value functions)

3. (Optional) Compute the equilibrium decision functions and value functions for McGrattan, Rogerson, and Wright (1997).



In [3]:
# Packages used
using ForwardDiff, LinearAlgebra, PrettyTables

In [ ]:
#= PS: Ellen said that the linear quadratic approximation is useful around the steady-state. 
This is important, since it won't be useful for cases with a lot of wiggles. =#

---
# Question 1

The following function takes two inputs: a function $f:\mathbb{R}^n \rightarrow \mathbb{R}$ and an $n$-dimensional vector $x$. It then computes the $n$-dimensional real Jacobian vector and the $n$ x $n$ dimensional real Hessian matrix.

In [4]:
function derivatives(f,x) 
    # Function take x and f as inputs; the output is the Jacobian (actually, the gradient, since it's a scalar real function) and Hessian matrices.
   
    # Since f is a scalar real value function, using ForwardDiff.jacobian is not appropriate, since it expects a vector of functions.
    J = ForwardDiff.gradient(f, x) 
    H = ForwardDiff.hessian(f, x)

    return J,H
end

derivatives (generic function with 1 method)

Let's test the function above using a simple example. Function $g$ is such that $g:\mathbb{R}^2 \rightarrow \mathbb{R}$.

In [5]:
# Create g:
g(z) = z[1]^3+z[2]^3

# Choose a 2-dimensional vector:
y = [1,1];

# Compute the Jacobian and Hessian:
out = derivatives(g,y);

# Print the results:
println("Gradient (Jacobian): ", out[1])
println("\nHessian: ", out[2])

Gradient (Jacobian): [3.0, 3.0]

Hessian: [6.0 0.0; 0.0 6.0]


# Question 2 - Direct Iteration

The following function takes $A,B,C,Q,R,W,\beta$ as inputs and produces $F,P$ as outputs using Direct Iteration. Note that while $\beta$ is not listed as an input in the Homework, it is necessary for performing some transformations.

In [5]:
function getPF_di(A,B,C,Q,R,W,β) 

    tA = sqrt(β)*(A-B*inv(R)*W'); #tA stands for ̃A; same holds for the transformations below
    tB = sqrt(β)*B;
    tQ = Q-W*inv(R)*W';

    sizeA = size(A); # get the dimensions of matrix A
    P_old = zeros(sizeA); # initial guess for Ricatti Matrix; note it is symmetric + negative semidefinite
    P_new = P_old; # setting a new matrix that will store the updates for P
    difP = 1; # initial value for the norm of the difference for P's iteration

    difF = difP; # initial value for the norm of the difference for F's iteration
    tF_old = inv(R + tB'*P_old*tB)*tB'*P_old*tA; # calculating an initial value for ̃F (tildeF); 
    # apparently, there's a typo in the paper, since the last ̃B should be transposed
    tF_new = tF_old; # setting a new matrix that will store the updates for ̃F

    # PS: Note that P_new stands for P_{n+1} while ̃F_new for ̃F_n.
    
    γ = 1e-8; # tolerance parameter γ. I'm setting γ ≡ γ_i ∀ i∈{1,2}
    tolP = γ; # initial tolerance for P
    tolF = tolP; # initial tolerance for ̃F

    while difP > tolP || difF > tolF # that is, loop will stop only when both difP < tolP AND difF < tolF
        tF_old = tF_new; # update F_old
        P_old = P_new; # update P_old
        P_new = tQ + tA'*P_old*tA - tA'*P_old*tB*inv(R + tB'*P_old*tB)*tB'*P_old*tA; # calculate P_new using step 1 from the paper
        tF_new = inv(R + tB'*P_old*tB)*tB'*P_old*tA; # calculate tF_new using step 1 from the paper

        difP = norm(P_new-P_old,Inf); # taking the supnorm
        difF = norm(tF_new-tF_old,Inf); 

        tolP = γ*norm(P_old,Inf); # update tolerance for P
        tolF = γ*norm(tF_old,Inf); # update tolerance for tF
    end

    P = P_new; # solution for P
    F = tF_new + inv(R)*W'; # solution for F

    return F,P
end

getPF_di (generic function with 1 method)

Let's test the function using the exercise in McGrattan(1990):

In [6]:
# Parameters
β = 0.95;
τ = 0.5;
α = 0.33;
ρ = 0.95;
kbar = (α*β/(1 - β))^(1/(1 - α));

# Matrices
A = [1 0 0; 0 1 0; 0 0 ρ];
B = [1,0,0];
C = [0 0 0; 0 0 0; 0 0 1];
aux = zeros(3,3);
aux[1,1] = (-τ*(α^2) + α^2 - α)/(kbar^2);
aux[1,2] = (τ*(α^2) - α^2 + 2*α)/kbar;
aux[1,3] = (α*(1 - τ))/kbar;
aux[2,1] = (τ*(α^2) - α^2 + 2*α)/kbar;
aux[2,2] = 2/(1 - τ) - 3*α + (α^2)*(1 - τ);
aux[2,3] = 1 + α*τ - α;
aux[3,1] = (α*(1 - τ))/kbar;
aux[3,2] = 1 + α*τ - α;
aux[3,3] = 1 - τ;
Q = ((kbar^(α*(1 - τ)))/2)*aux;
W = [α*τ*(kbar)^(-α*τ - 1),-(1+α*τ)*(kbar)^(-α*τ),τ*(kbar)^(-α*τ)]/2;
R = (-τ*(kbar)^(-α*τ - α))/2;

In [7]:
time_di = @elapsed begin
F_di,P_di = getPF_di(A,B,C,Q,R,W,β)
end 

# Print the solution:
pretty_table(round.(F_di;digits=4); header = ["1","2","3"],
            title = "Vector F",row_labels = ["1"],
            border_crayon = crayon"bold yellow",
            tf = tf_simple)

pretty_table(round.(P_di;digits=4); header = ["1","2","3"],
            title = "\n \nMatrix P",
            row_labels = ["1","2","3"],
            border_crayon = crayon"bold yellow",
            tf = tf_simple)

Vector F
==== ======== ======== ==========
           1        2         3  
==== ======== ======== ==========
  1   0.0765   -1.184   -1.8872  
==== ======== ======== ==========

 
Matrix P
==== ========= ========= ==========
            1         2         3  
==== ========= ========= ==========
  1   -0.0095    0.4816   -0.0228  
  2    0.4816   50.2189    8.4119  
  3   -0.0228    8.4119    4.5879  
==== ========= ========= ==========


# Question 2 - Vaughan's Algorithm

The following function takes $A,B,C,Q,R,W,\beta$ as inputs and produces $F,P$ as outputs using Vaughan's Algorithm. Note that while $\beta$ is not listed as an input in the Homework, it is necessary for performing some transformations.

In [15]:
function getPF_va(A,B,C,Q,R,W,β)
    
    tA = sqrt(β)*(A-B*inv(R)*W'); #tA stands for ̃A; same holds for the transformations below
    tB = sqrt(β)*B;
    tQ = Q-W*inv(R)*W';

    # Construct elements that will be in the Hamiltonian matrix:
    a11 = inv(tA);
    a12 = inv(tA)*tB*inv(R)*tB';
    a21 = tQ*inv(tA);
    a22 = tQ*inv(tA)*tB*inv(R)*tB' + tA';

    # Building the Hamiltonian matrix:
    ha = [a11 a12;a21 a22];

    # Get the eigenvalues and eigenvectors of the Hamiltonian matrix:
    eg = eigen(ha); 

    # Sort eigenvalues in descending order (in case they are not automatically ordered in that way)
    sort_idx = sortperm(eg.values, rev=true);

    # Get the eigenvalues in descending order
    sort_egval = eg.values[sort_idx];

    # Get the eigenvectors corresponding to the eigenvalues that lie outside the unit circle
    sort_egvec = eg.vectors[:, sort_idx];

    rows = size(a11, 1);  # number of rows in a11
    cols = size(a11, 2); # number of columns in a11

    V11 = sort_egvec[1:rows, 1:cols]; # compute V_11        
    V21 = sort_egvec[rows+1:end, 1:cols]; # compute V_21
    
    # Compute P and F
    P = V21*inv(V11); # solution for P
    F = inv(R .+ tB'*P*tB)*tB'*P*tA .+ inv(R)*W'; # solution for F

    return F,P

end

getPF_va (generic function with 1 method)

Let's test the function using the exercise in McGrattan(1990):

In [22]:
# Parameters
β = 0.95;
τ = 0.5;
α = 0.33;
ρ = 0.95;
kbar = (α*β/(1 - β))^(1/(1 - α));

# Matrices
A = [1 0 0; 0 1 0; 0 0 ρ];
B = [1,0,0];
C = [0 0 0; 0 0 0; 0 0 1];
aux = zeros(3,3);
aux[1,1] = (-τ*(α^2) + α^2 - α)/(kbar^2);
aux[1,2] = (τ*(α^2) - α^2 + 2*α)/kbar;
aux[1,3] = (α*(1 - τ))/kbar;
aux[2,1] = (τ*(α^2) - α^2 + 2*α)/kbar;
aux[2,2] = 2/(1 - τ) - 3*α + (α^2)*(1 - τ);
aux[2,3] = 1 + α*τ - α;
aux[3,1] = (α*(1 - τ))/kbar;
aux[3,2] = 1 + α*τ - α;
aux[3,3] = 1 - τ;
Q = ((kbar^(α*(1 - τ)))/2)*aux;
W = [α*τ*(kbar)^(-α*τ - 1),-(1+α*τ)*(kbar)^(-α*τ),τ*(kbar)^(-α*τ)]/2;
R = (-τ*(kbar)^(-α*τ - α))/2;

In [17]:
time_va = @elapsed begin
F_va,P_va = getPF_va(A,B,C,Q,R,W,β)
end

# Print the solution:
pretty_table(round.(F_va;digits=4); header = ["1","2","3"],
            title = "Vector F",
            border_crayon = crayon"bold yellow",
            tf = tf_simple)

pretty_table(round.(P_va;digits=4); header = ["1","2","3"],
            title = "\n \n Matrix P",
            row_labels = ["1","2","3"],
            border_crayon = crayon"bold yellow",
            tf = tf_simple)

Vector F
========= ======== ==========
       1        2         3  
========= ======== ==========
  0.0765   -1.184   -1.8872  
========= ======== ==========

 
 Matrix P
==== ========= ========= ==========
            1         2         3  
==== ========= ========= ==========
  1   -0.0095    0.4816   -0.0228  
  2    0.4816   50.2189    8.4119  
  3   -0.0228    8.4119    4.5879  
==== ========= ========= ==========


It's useful to check whether the outputs from the Direct Iteration Method and Vaughan's Algorithm coincide. Computationally, it’s expected that the solutions will not be exactly the same, but they should be close. To compare the solutions, I use a tolerance of $10^{-6}$.

In [14]:
if isapprox(F_di, F_va, rtol=1e-6) && isapprox(P_di, P_va, rtol=1e-6)
    println("Solution for Direct Iteration and Vaughan's Algorithm coincide.")
else
    println("Solutions for Direct Iteration and Vaughan's Algorithm are different.")
end

Solution for Direct Iteration and Vaughan's Algorithm coincide.


Lastly, let's compare the time it took for each method to reach a solution:

In [15]:
println("Simulation via Direct Iteration: $(round(time_di, digits=4)) seconds.")
println("Simulation via Vaughan's Algorithm: $(round(time_va, digits=4)) seconds.")
println("Direct Iteration was approximately $(round(time_va/time_di, digits=2)) times faster than Vaughan's Algorithm.")

Simulation via Direct Iteration: 1.2396 seconds.
Simulation via Vaughan's Algorithm: 2.7167 seconds.
Direct Iteration was approximately 2.19 times faster than Vaughan's Algorithm.


As we can see above, although both algorithms quickly reach a solution, Direct Iteration was more efficient. This result surprised me for two reasons. First, matrix operations are typically computed faster than iterative methods. Second, this finding contrasts with the results reported in the paper.

One possible explanation is that one of the functions I used in Vaughan's algorithm might be computationally demanding.